In [1]:
import pandas as pd

reviews_raw: pd.DataFrame = pd.read_csv("./data/letterboxd_250movie_reviews.csv")

reviews_raw.head()
valid_reviews = reviews_raw[reviews_raw['Rating'].notna()][reviews_raw['Status'] == 'Watched'][~reviews_raw['Review'].isna()].sample(frac=1, random_state=42)
valid_reviews.count()

/var/folders/6g/d16gbw5x5z9fk0rmvxw5z5q80000gn/T/ipykernel_8476/4114284106.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  valid_reviews = reviews_raw[reviews_raw['Rating'].notna()][reviews_raw['Status'] == 'Watched'][~reviews_raw['Review'].isna()].sample(frac=1, random_state=42)
/var/folders/6g/d16gbw5x5z9fk0rmvxw5z5q80000gn/T/ipykernel_8476/4114284106.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  valid_reviews = reviews_raw[reviews_raw['Rating'].notna()][reviews_raw['Status'] == 'Watched'][~reviews_raw['Review'].isna()].sample(frac=1, random_state=42)


Unnamed: 0    61096
Review        61096
Rating        61096
Date          61096
Status        61096
Movie         61096
dtype: int64

In [2]:
valid_reviews.head()

,Unnamed: 0,Review,Rating,Date,Status,Movie
43340,43340,A scathing political thriller from the great C...,★★★★★,19 Jul 2022,Watched,z
29334,29334,It's impossible to not respect how kids of a c...,★★★★½,04 Aug 2021,Watched,where-is-the-friends-house
72718,72718,"""There is only one golden rule: ""Do unto other...",★★★★★,21 Oct 2023,Watched,eureka
87904,87904,This is what I expect from screenplay when I s...,★★★★★,02 Jan 2022,Watched,happy-together-1997
57955,57955,"#madlad Mifune'If you would shed blood, then l...",★★★★,13 Apr 2021,Watched,throne-of-blood


remove non-english reviews:

In [3]:
from langdetect import detect, DetectorFactory
from langdetect.lang_detect_exception import LangDetectException

# Set seed for consistent results
DetectorFactory.seed = 0

def is_english(text: str) -> bool:
    try:
        return detect(text) == 'en'
    except LangDetectException:
        # Handle very short texts or detection failures
        return False

is_english(valid_reviews['Review'].iloc[0])
valid_reviews = valid_reviews[valid_reviews['Review'].apply(is_english)]

normalize the rating field:

In [4]:
valid_reviews.count()

valid_reviews.groupby('Rating').count()

def normalize_rating(rating: str) -> str:
    mapping = {
        '½': '0.5',
        '★': '1',
        '★½': '1.5',
        '★★': '2',
        '★★½': '2.5',
        '★★★': '3',
        '★★★½': '3.5',
        '★★★★': '4',
        '★★★★½': '4.5',
        '★★★★★': '5'
    }
    return f'{mapping[rating]} stars'

valid_reviews['Rating'] = valid_reviews['Rating'].apply(normalize_rating)
valid_reviews.head()

,Unnamed: 0,Review,Rating,Date,Status,Movie
43340,43340,A scathing political thriller from the great C...,5 stars,19 Jul 2022,Watched,z
29334,29334,It's impossible to not respect how kids of a c...,4.5 stars,04 Aug 2021,Watched,where-is-the-friends-house
72718,72718,"""There is only one golden rule: ""Do unto other...",5 stars,21 Oct 2023,Watched,eureka
87904,87904,This is what I expect from screenplay when I s...,5 stars,02 Jan 2022,Watched,happy-together-1997
57955,57955,"#madlad Mifune'If you would shed blood, then l...",4 stars,13 Apr 2021,Watched,throne-of-blood


sample 200 reviews for each rating:

In [5]:
sample_for_each_rating = 200
sampled_valid_reviews = pd.concat([valid_reviews[valid_reviews['Rating'] == rating].sample(n = sample_for_each_rating, random_state=42) for rating in valid_reviews['Rating'].unique()])
# remove 'Watched' column
sampled_valid_reviews = sampled_valid_reviews.drop(columns=['Status','Date']).rename(columns={'Unnamed: 0': 'Original_ID'})
sampled_valid_reviews.head()

,Original_ID,Review,Rating,Movie
26598,26598,Satantango is a pure work of the devil which c...,5 stars,satantango
64304,64304,Best Actress Winners Finished (Complete Rankin...,5 stars,whos-afraid-of-virginia-woolf
21703,21703,This my family’s Christmas movie growing up an...,5 stars,its-a-wonderful-life
70342,70342,"""All this happiness that you wished for me has...",5 stars,the-tale-of-the-princess-kaguya
73915,73915,shelley duvall deserves the praise that jack n...,5 stars,the-shining


see some of the reviews:

In [6]:
def summarize(df_row)-> str:
    return f"Review: {df_row['Review']} - Movie: {df_row['Movie']}"

def check_rating(rating):
    return sampled_valid_reviews[sampled_valid_reviews['Rating'] == rating][sampled_valid_reviews['Review'].str.len() < 250].apply(summarize, axis=1).head().tolist()

def log_sample_for_rating(rating):
    samples = check_rating(rating)
    print(f"Samples for rating {rating}:")
    for sample in samples:
        print(f' {sample}')
    print("\n")
log_sample_for_rating('5 stars')
log_sample_for_rating('2.5 stars')
log_sample_for_rating('0.5 stars')

Samples for rating 5 stars:
 Review: shelley duvall deserves the praise that jack nicholson gets for this film. - Movie: the-shining
 Review: Well, I feel devastated after finishing this. The last 30 minutes were so sad to watch.Edit: I just noticed there are more films that continues Antoine's story, I guess I will be watching all of those soon. - Movie: the-400-blows
 Review: It's just a movie; it won't hurt you.The movie: - Movie: landscape-in-the-mist
 Review: okay but like why is this the greatest movie ever made in film history!! - Movie: whiplash-2014
 Review: You are not machines!You are not cattle!You are men!You have the love of humanity in your hearts!You don't hate!Only the unloved hate - the unloved and the unnatural!Love this man. That's all.Charlie Chaplin movies (Watched & Ranked) - Movie: the-great-dictator


Samples for rating 2.5 stars:
 Review: Letterboxd Top 250 Marathon: 54/130The first act is masterful. - Movie: farewell-my-concubine
 Review: Beautifully imaginat

/var/folders/6g/d16gbw5x5z9fk0rmvxw5z5q80000gn/T/ipykernel_8476/1197829163.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  return sampled_valid_reviews[sampled_valid_reviews['Rating'] == rating][sampled_valid_reviews['Review'].str.len() < 250].apply(summarize, axis=1).head().tolist()
/var/folders/6g/d16gbw5x5z9fk0rmvxw5z5q80000gn/T/ipykernel_8476/1197829163.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  return sampled_valid_reviews[sampled_valid_reviews['Rating'] == rating][sampled_valid_reviews['Review'].str.len() < 250].apply(summarize, axis=1).head().tolist()
/var/folders/6g/d16gbw5x5z9fk0rmvxw5z5q80000gn/T/ipykernel_8476/1197829163.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  return sampled_valid_reviews[sampled_valid_reviews['Rating'] == rating][sampled_valid_reviews['Review'].str.len() < 250].apply(summarize, axis=1).head().tolist()


In [7]:
# save the sampled reviews to a new CSV file
sampled_valid_reviews.to_csv('./letterboxd_250movie_reviews_normalized_sampled.csv', index=False)